# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
## when sql_queries.py will be changed, stop the kernell, start again to get the changes and run again
from sql_queries import *
import datetime

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
def read_files(filepath):
    df = pd.read_json(filepath[0], lines=True)
    for i in range(1,len(filepath)):
        df = pd.concat([df, pd.read_json(filepath[i], lines=True)])
    return df    

In [6]:
song_files = 'data/song_data/'
filepath = get_files(song_files)

df = read_files(filepath)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
0,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
0,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
0,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
0,ARKFYS91187B98E58F,NaN,,NaN,Jeff And Sheri Easter,267.70240,1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),0
0,ARD0S291187B9B7BF5,NaN,Ohio,NaN,Rated R,114.78159,1,SOMJBYD12A6D4F8557,Keepin It Real (Skit),0
0,ARGSJW91187B9B1D6B,35.21962,North Carolina,-80.01955,JennyAnyKind,218.77506,1,SOQHXMF12AB0182363,Young Boy Blues,0
0,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982
0,ARXR32B1187FB57099,NaN,,NaN,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007


In [7]:
## find out datatypes to songs and artisti tables

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 0
Data columns (total 10 columns):
artist_id           75 non-null object
artist_latitude     34 non-null float64
artist_location     75 non-null object
artist_longitude    34 non-null float64
artist_name         75 non-null object
duration            75 non-null float64
num_songs           75 non-null int64
song_id             75 non-null object
title               75 non-null object
year                75 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 6.4+ KB


In [8]:
##view first records
df.head()


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
0,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
0,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
0,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
## select first record to song table
## convert to list
## convert from numpy numeric type to python

song_data = df[['song_id','title','artist_id', 'year', 'duration']].values
song_data = [x if isinstance(x, str) else x.tolist() for x in song_data]  
song_data = [x if str(x) != 'nan' else None for x in song_data]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
for row in song_data:
    if not(row[0] == None) and not(row[1] == None):
        cur.execute(song_table_insert, row)
   
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_list = list(df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values)
artist_list = [x if isinstance(x, str) else x.tolist() for x in artist_list]
artist_list = [x if str(x) != 'nan' else None for x in artist_list]
               

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
for row in artist_list:
    if not(row[0] == None)  and  not(row[1] == None):
        cur.execute(artist_table_insert, row)
        
conn.commit()        
    

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = "data/log_data"
filepath = get_files(log_files)
df_log = read_files(filepath)

In [14]:
df_log.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df_time = pd.to_datetime(df_log[df_log['page'] =='NextSong']['ts'], unit='ms')
#df_time = df_time[:20]
dict_time = {}
dict_time['timestamp'] = df_time
dict_time['hour'] = pd.DatetimeIndex(df_time).time
dict_time['day'] = pd.DatetimeIndex(df_time).day
dict_time['week'] = pd.DatetimeIndex(df_time).week
dict_time['month'] = pd.DatetimeIndex(df_time).month
dict_time['year'] = pd.DatetimeIndex(df_time).year
dict_time['weekday'] = df_time.dt.dayofweek + 1
dict_time['day_name'] = df_time.dt.day_name()

time_df = pd.DataFrame.from_dict(dict_time)
time_df.head()




,timestamp,hour,day,week,month,year,weekday,day_name
0,2018-11-30 00:22:07.796,00:22:07.796000,30,48,11,2018,5,Friday
1,2018-11-30 01:08:41.796,01:08:41.796000,30,48,11,2018,5,Friday
2,2018-11-30 01:12:48.796,01:12:48.796000,30,48,11,2018,5,Friday
3,2018-11-30 01:17:05.796,01:17:05.796000,30,48,11,2018,5,Friday
4,2018-11-30 01:20:56.796,01:20:56.796000,30,48,11,2018,5,Friday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [16]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [17]:
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [18]:
df_log.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
user_table_insert = """INSERT INTO users as us
                      (user_id, first_name, last_name, gender, level) 
                      values (%s, %s, %s, %s, %s)
                      ON CONFLICT (user_id) DO UPDATE 
                      SET level = COALESCE(EXCLUDED.level, us.level)"""                     

In [23]:
conn.commit()

In [33]:
user_df = df_log[['userId','firstName', 'lastName', 'gender', 'level']]
for i, row in user_df.iterrows():
    if str(row.userId).isnumeric(): 
        cur.execute(user_table_insert, row)
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [20]:
df.columns, df_log.columns

(Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
        'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
       dtype='object'),
 Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
        'length', 'level', 'location', 'method', 'page', 'registration',
        'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
       dtype='object'))

In [50]:
df.merge(df_log, left_on=['title', 'duration'], right_on=['song', 'length'])[['artist_name', 'artist_id', 'song_id']]


,artist_name,artist_id,song_id
0,Elena,AR5KOSW1187FB35FF4,SOZCTXZ12AB0182364


In [21]:
songplay_table_insert = """INSERT INTO songsplay 
        (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
        ON CONFLICT DO NOTHING;"""
                        

In [22]:
for index, row in df_log.iterrows():   
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.length, row.song, row.artist))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    time = pd.to_datetime(row.ts, unit='ms')
    
    # insert songplay record    
    if str(row.userId).isnumeric()  and str(row.ts).isnumeric():
        songplay_data = (time, int(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent,)
        #print (songplay_data)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()

In [23]:
conn.commit()

In [24]:
 conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
cur.close()
conn.close()

In [26]:
cur.close()
conn.close()

conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.